## **efficient net + triplet**

구조 커스텀에 용이하기 위해 파이토치로 만들기

In [1]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


## **1.모델**

필요 라이브러리 설치

In [2]:
import torch

#신경망(Neural Network) 관련 기능을 제공하는 모듈. 
#레이어(layer), 활성화 함수(activation), 손실 함수(loss), 신경망 구성 요소 포함
import torch.nn as nn

import torchvision.models as models

#코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

efficient net -> 임베딩 벡터 변환 함수

In [ ]:
class EfficientNetEmbedding(nn.Module):
    def __init__(self, embedding_size=128):
        super().__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)  # 사전학습 EfficientNet. imagenet 사용(논문과동일)
        self.features = self.base_model.features  # 분류기(fc) 제거, feature extractor 부분만 사용
        self.pool = nn.AdaptiveAvgPool2d(1)  # 마지막 feature map에 global average pooling
                                            # feature map을 한줄 벡터로 압축해야해서...
                                            
        self.embedding = nn.Linear(1280, embedding_size)  # 1280채널 → 임베딩 크기(128)로 축소
        self.l2_norm = nn.functional.normalize  # 임베딩 벡터 정규화 함수

    def forward(self, x):
        x = self.features(x)  # 이미지 특징 추출
        x = self.pool(x)  # 채널별 평균값으로 차원 축소
        x = torch.flatten(x, 1)  # 2D → 1D 벡터로 변환
        x = self.embedding(x)  # 임베딩 벡터 생성
        x = self.l2_norm(x, dim=1)  # 임베딩 벡터 정규화 (길이 1로)
        return x


In [ ]:
model = EfficientNetEmbedding(embedding_size=128)

loss_fn = nn.TripletMarginLoss(margin=1.0)  # Triplet Loss 함수 (margin은 거리 차이 최소 기준)
#앵커와 음성 간 거리가 앵커와 양성 간 거리보다 최소 1.0 이상 더 커야
#손실이 0이 되고 학습이 멈춤 (조건 만족)
#만약 두 거리 차이가 margin보다 작으면 손실이 양수이고, 모델은 차이를 늘리려고 학습함

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

c:\Users\Administrator\.conda\envs\tf-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\.conda\envs\tf-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [01:34<00:00, 228kB/s] 


triplet loss 함수

In [ ]:
def train_triplet(model, data_loader, optimizer, loss_fn, device):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0  # 손실 합산용 변수 초기화
    
    for anchor, positive, negative in data_loader:  # 데이터로더에서 triplet 배치 단위로 불러오기
        anchor = anchor.to(device)      # 앵커 이미지 배치를 GPU/CPU에 올림
        positive = positive.to(device)  # 양성 이미지 배치를 GPU/CPU에 올림
        negative = negative.to(device)  # 음성 이미지 배치를 GPU/CPU에 올림

        optimizer.zero_grad()  # 이전 배치의 기울기 초기화
        
        # 각 배치에 대해 임베딩 벡터 생성
        anchor_embed = model(anchor)      
        positive_embed = model(positive)
        negative_embed = model(negative)
        
        # Triplet Loss 계산 (앵커-양성은 가깝게, 앵커-음성은 멀게)
        loss = loss_fn(anchor_embed, positive_embed, negative_embed)
        
        # 손실값을 기준으로 역전파 (모델 가중치 업데이트 방향 계산)
        loss.backward()
        
        # 옵티마이저로 가중치 업데이트
        optimizer.step()
        
        # 배치 손실값을 누적
        total_loss += loss.item()
    
    # 전체 데이터셋 평균 손실값 반환
    return total_loss / len(data_loader)



데이터를 batch 단위로 받아서 EfficientNet 모델로 임베딩 벡터를 뽑고

Triplet Loss 함수로 거리 차이를 계산해서 손실값을 구함

역전파로 모델 파라미터를 업데이트

## **2.갤러리 임베딩 추출**

In [ ]:
def extract_embedding(model, data_loader, device):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for x in data_loader:
            x = x.to(device)
            emb = model(x)
            embeddings.append(emb.cpu().numpy())
    return np.vstack(embeddings)


## **3.테스트 임베딩 추출**

## **4.top-k 추천**